# Defining a modifier function

In [1]:
def modify(frame, data):


IndentationError: expected an indented block (1777889733.py, line 1)

In [2]:
def my_mod_function(frame, data):
    ...
    ...

# Insert the modifier function into a pipeline:
pipeline.modifiers.append(my_mod_function)

# Pipeline evaluation: The system will invoke your user-defined function.
data = pipeline.compute()

NameError: name 'pipeline' is not defined

# Implementing a modifier function

## Writing well-behaved modifier functions

In [3]:
total_energy = 0.0  # A global variable accessed below

def modify(time, data):

    # Do NOT add modifiers to the current pipeline (because any changes made to a pipeline
    # while it is being evaluated will lead to an infinite loop):
   ovito.scene.selected_pipeline.modifiers.append(...)

    # Do NOT insert new objects into the current scene (because a modifier function is only
    # supposed to modify the data collection flowing down the pipeline):
    pipeline2 = import_file(...)
   pipeline2.add_to_scene()

    # Do NOT modify global variables or objects (because the system may call your
    # function an arbitrary number of times):
   total_energy += numpy.sum(data.particles['Potential Energy'])

    # Do NOT perform file I/O from within a modifier function because you don't know when
    # and how often the pipeline system is going to call your function:
   file = open('my_output.txt', 'w')

IndentationError: unexpected indent (469203770.py, line 11)

## How to save computed information to disk

In [4]:
pipeline = import_file(...)
for frame in range(pipeline.source.num_frames):
    data = pipeline.compute(frame)
    ...
    # Perform analysis of the current frame...
    ...
    # Write results to an output file using Python functions
    file = open('output_file.%i.txt' % frame, 'w')
    ...

NameError: name 'import_file' is not defined

## One-time loading of input data

In [5]:
from ovito.io import import_file
from ovito.pipeline import FileSource

# Load the reference particle configuration in a one-time initialization step
# outside of the modifier function:
reference = FileSource()
reference.load("input/simulation.0.dump", sort_particles=True)

def modify(frame, data):
    displacements = data.particles.positions - reference.data.particles.positions
    data.particles_.create_property('Displacement', data=displacements)

ModuleNotFoundError: No module named 'ovito'

## Long-running modifier functions

In [6]:
def modify(frame, data):
    # This is a long-running loop over all particles in the system:
    for i in range(data.particles.count):
        # Perform one short computational step:
        ...
        # Temporarily yield control to the system to process input events:
        yield

In [7]:
def modify(frame, data):
    for i in range(data.particles.count):
        ...
        yield (i / data.particles.count)

In [8]:
def modify(frame, data):
    yield "Performing an analysis..."
    ...